In [1]:
# !pip install wandb optuna
import typing as t
from pathlib import Path
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam, AdamW, Optimizer
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import dataclasses
import optuna
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import types
from typing import Callable
import wandb

PROJECT = "whitebox-inversion"

/home/vedang/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

@dataclasses.dataclass
class Config:
    # Core
    prefix: str = "Hello! Will you marry me?"
    suffix_length: int = 13
    seed: int = 42*69*69
    batch_size: int = 500
    randomise: bool = False
    add_eos: bool = False
    relax_hot_val: float = 0.01
    model_id = "gpt2"

  # Learning
    learning_rate: float = 2e-3
    iterations: int = 500
    optimizer: str = "adam"
    scheduler_t_0: int = 28

    # Entropy projection
    start_entropy: float = 1.
    stop_entropy: float = 1.

    # Re-initialization
    reinit_threshold: int = 0
    reinit_rand_alpha: float = 1e-4
    reinit_blend_alpha: float = 1e-2

    # Blending
    best_blend_alpha: float = 0
    best_blend_threshold: float = 0.05

    # Discrete sampling
    discrete_sampling_temp: float = 2.0

    # Optuna
    use_optuna: bool = False
    optuna_trials: int = 10
    optuna_storage: str = "sqlite:///optuna.db"
    optuna_study_name: str = PROJECT

    # Wandb
    wandb_logging: bool = True

def adapt_for_optuna(config: Config, trial: optuna.Trial) -> Config:
    config.wandb_logging = False
    config.suffix_length = trial.suggest_int("suffix_length", 1, 30)
    config.relax_hot_val = trial.suggest_float("relax_hot_val", 0.001, 0.1)
    config.learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    config.optimizer = trial.suggest_categorical("optimizer", ["adam", "adamw"])
    config.scheduler_t_0 = trial.suggest_int("scheduler_t_0", 5, 30)
    config.stop_entropy = trial.suggest_float("stop_entropy", 0.99, 1.0)
    config.reinit_threshold = trial.suggest_int("reinit_threshold", 0, 300, step=10)
    config.best_blend_alpha = trial.suggest_float("best_blend_alpha", 0, 0.1)
    config.best_blend_threshold = trial.suggest_float("best_blend_threshold", 0, 0.1)
    config.discrete_sampling_temp = trial.suggest_float("discrete_sampling_temp", 1.0, 3.0)
    return config


In [3]:
def get_model_wrapper(model_id: str, maximum_layer: int = None, keep_head: bool = False, device="cuda") -> nn.Module:
    if maximum_layer is not None:
        return truncate_model(model_id, maximum_layer, keep_head, device), \
            AutoTokenizer.from_pretrained(model_id)
    else:
        return AutoModelForCausalLM.from_pretrained(model_id).to(device=device), \
            AutoTokenizer.from_pretrained(model_id)

def truncate_model(model_id: str, maximum_layer: int, keep_head: bool = False, device="cuda") -> nn.Module:
    '''This truncates the LM up to the target layer, meaning
    that inference is faster. As such the output of model.generate
    will be the desired activation layer'''
    model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

    if hasattr(model, 'transformer') and hasattr(model.transformer, 'h'):
        # Truncate the list of layers
        model.transformer.h = nn.ModuleList(model.transformer.h[:maximum_layer + 1])

        # Adjust the config to reflect the new number of layers
        if hasattr(model.config, 'num_hidden_layers'):
            model.config.num_hidden_layers = maximum_layer + 1

        if not keep_head:
             # Remove the language model head if desired
            if hasattr(model, 'lm_head'):
                delattr(model, 'lm_head')

            # The final layer norm seems important to smooth running
            # however, we can replace it with an identity function
            if hasattr(model.transformer, 'ln_f'):
                model.transformer.ln_f = nn.Identity(model.transformer.ln_f.normalized_shape)

        # Modify the forward method to return the output of the last transformer block
        def new_forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=None, output_hidden_states=None, return_dict=None):
            outputs = self.transformer(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
            return outputs[0]  # Return the last hidden state directly

        model.forward = types.MethodType(new_forward, model)
    else:
        raise ValueError("Model structure not as expected. Please check the model architecture.")

    return model

In [4]:
def maximise_entropy_distribution_of_layer(activations, numerical_safety=1e-4):
    # Our goal is to get the layer to uniformally (>0) fire
    # by taking the abs, we get a worst case normalisation
    normaliser = torch.abs(activations).sum(dim=1).unsqueeze(1)
    # take a relu to only get positive values; normalising it
    # in this way promotes anti-sparsity
    p_activations = (torch.nn.functional.relu(activations) + 1e-4) / normaliser
    log_p_activations = torch.log(p_activations)
    losses = torch.nn.functional.kl_div(log_p_activations, \
                               (torch.ones(p_activations.shape[1])/p_activations.shape[1]).to(activations.device), \
                                        reduction='none') # preserves batch dims

    return losses.sum(dim=1)

def return_maximise_layer_entropy_distribution(numerical_safety=1e-4):
    _loss_wraper = lambda activations: maximise_entropy_distribution_of_layer(activations, numerical_safety=numerical_safety)
    return _loss_wraper

def maximise_entropy_distribution_of_logits(activations, unembedding_weights, numerical_safety=1e-4):
    # This method is similar to the activation layer optimisation
    # but this time we are deliberately trying to make it such that the logits
    # are maximally activated
    logits = (unembedding_weights@activations.T).T
    # by taking the abs, we get a worst case normalisation
    normaliser = torch.abs(logits).sum()
    # take a relu to only get positive values
    p_activations = (torch.nn.functional.relu(logits) + 1e-4) / normaliser
    log_p_activations = torch.log(p_activations)
    losses = torch.nn.functional.kl_div(log_p_activations, \
                               (torch.ones(p_activations.shape[1])/p_activations.shape[1]).to(activations.device), \
                                        reduction='none') # preserves batch dims

    return losses.sum(dim=1)

def return_maximise_logit_entropy_distribution(model_id, numerical_safety=1e-4, device="cuda"):
    _model, _ = get_model_wrapper(model_id, device=device)
    unembedding_weights = _model.lm_head.weight
    _loss_wraper = lambda activations: maximise_entropy_distribution_of_logits(activations, unembedding_weights, numerical_safety=numerical_safety)
    return _loss_wraper

In [5]:
def to_relaxed_one_hot(tokens: torch.Tensor, vocab_size: int, hot_val: float = 1.0) -> torch.Tensor:
    batch_size, seq_len = tokens.size()
    one_hot = torch.zeros(batch_size, seq_len, vocab_size, device=tokens.device)
    one_hot.scatter_(2, tokens.unsqueeze(-1).to(torch.int64), hot_val)
    remaining_prob = hot_val / (vocab_size - 1)
    one_hot += remaining_prob * (1 - one_hot)
    return one_hot.to(tokens.device)

def simplex_projection(tensor: torch.Tensor) -> torch.Tensor:
    d_batch, d_tokens, d_vocab = tensor.shape
    mu, _ = torch.sort(tensor, descending=True, dim=-1)
    cumulative = mu.cumsum(dim=-1)
    indices = torch.arange(1, d_vocab + 1, device=tensor.device).expand(d_batch, d_tokens, -1).float()
    threshold = (cumulative - 1) / indices
    rho = (mu > threshold).sum(dim=-1) - 1

    # Ensure rho doesn't go out of bounds
    rho = torch.clamp(rho, 0, d_vocab - 1)

    # Calculate threshold for each row
    threshold_per_row = torch.gather(threshold, 2, rho.unsqueeze(-1))  # Shape: [batch_size, seq_len, 1]
    projected = torch.max(tensor - threshold_per_row, torch.zeros_like(tensor))
    return projected

def entropy_projection(tensor: torch.Tensor, entropy: float) -> torch.Tensor:
    positive_mask = (tensor > 0).float()
    positive_count = positive_mask.sum(dim=-1, keepdim=True)
    c = positive_mask / positive_count
    R = torch.sqrt(1 - entropy - 1 / (positive_count))
    if R.isnan().any():
        return tensor
    norm_s_c = torch.norm(tensor - c, dim=-1, keepdim=True)
    needs_projection = (norm_s_c < R).float()
    does_not_need_projection = 1 - needs_projection
    scaled_s = torch.where(needs_projection.bool(), (R / norm_s_c) * (tensor - c) + c, tensor)
    projection = simplex_projection(scaled_s)
    result = does_not_need_projection * tensor + needs_projection * projection
    return result


In [6]:
def attack(model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, config: Config, loss_func: Callable, verbose: bool = False, discrete_loss_sample_rate: int = 1) -> t.Tuple[float, torch.Tensor]:
    device = next(model.parameters()).device
    optimizer: Optimizer

    if config.optimizer == "adamw":
        optimizer = AdamW([torch.zeros(1, device=device)], lr=config.learning_rate)
    elif config.optimizer == "adam":
        optimizer = Adam([torch.zeros(1, device=device)], lr=config.learning_rate)
    else:
        raise ValueError(f"Invalid optimizer: {config.optimizer}")

    prefix_str = config.prefix if config.prefix else ""

    suffix_tokens = torch.randint(low=0, high=tokenizer.vocab_size, size=(config.batch_size, config.suffix_length), device="cpu")
    prefix_tokens = tokenizer.encode(prefix_str, return_tensors='pt').squeeze(0).repeat(config.batch_size, 1).to("cpu")

    all_tokens = torch.cat([prefix_tokens, suffix_tokens], dim=1).to(device)
    suffix_slice = slice(prefix_tokens.shape[1], all_tokens.shape[1])

    labels = all_tokens.clone().type(torch.int64)
    labels[:, :suffix_slice.stop] = -100 # signals to be ignored
    labels = labels[:,1:].flatten(start_dim=0,end_dim=1) # ready for CE loss

    inputs = to_relaxed_one_hot(all_tokens, tokenizer.vocab_size, hot_val=config.relax_hot_val)
    if config.randomise:
        # extra randomness that is NOT limited to the token plane
        random_values = torch.rand_like(inputs[:,suffix_slice])
        normalized_values = random_values / random_values.sum(dim=-1, keepdim=True)
        inputs[:,suffix_slice] = normalized_values

    inputs.requires_grad_()

    optimizer.param_groups.clear()
    optimizer.add_param_group({"params": [inputs]})

    # Collect relevant dimensions assuming inputs has shape
    # [batch_size, sequence_length, vocab_size]
    batch_size, prediction_seq_length, vocab_size = config.batch_size, \
    inputs.shape[1] - 1, tokenizer.vocab_size

    scheduler = CosineAnnealingLR(optimizer, config.scheduler_t_0)

    best_loss = torch.inf * torch.ones(1, device=device)
    best_discrete = None
    current_entropy = config.start_entropy
    entropy_delta = (config.stop_entropy - config.start_entropy) / config.iterations

    for i in tqdm(range(1, config.iterations + 1)):
        input_embeds = (inputs @ model.transformer.wte.weight).to(device)
        # Assuming input_embeds has shape [batch_size, sequence_length, embedding_dim]
        # we take the final
        activations = (model(inputs_embeds=input_embeds))[:, -1, :]
        # Calculate loss
        loss = loss_func(activations)

        optimizer.zero_grad()
        dummy_grad = torch.ones_like(loss)
        loss.backward(gradient=dummy_grad)

        inputs.grad.data[:, : suffix_slice.start] = 0
        inputs.grad.data[:, suffix_slice.stop :] = 0

        optimizer.step()
        scheduler.step()

        inputs.data[:, suffix_slice] = simplex_projection(inputs.data[:, suffix_slice])
        if current_entropy != 1.0:
            inputs.data[:, suffix_slice] = entropy_projection(inputs.data[:, suffix_slice], current_entropy)
        current_entropy += entropy_delta
        discrete = torch.argmax(inputs.data[:, suffix_slice], dim=2)
        all_tokens[:, suffix_slice] = discrete
        if verbose or config.wandb_logging:
          save_loss = loss.detach().cpu().mean()
        del activations, loss # release memory
        # collect and potentially print outputs
        if i % discrete_loss_sample_rate == 0:
            with torch.no_grad():
                activations_discrete = (model(all_tokens))[:, -1]
                discrete_loss = loss_func(activations_discrete)
                avg_loss_i, avg_best_loss = discrete_loss.mean(), best_loss.mean()
                if avg_loss_i < avg_best_loss:
                    best_loss = discrete_loss
                    best_discrete = discrete
            if verbose:
                current_discrete_text = [tokenizer.decode(x) for x in discrete[:3]]
                print(f"[{i}] L-rel: {save_loss:.5f} / L-dis: {discrete_loss.flatten().mean():.5f} / Best: {best_loss.flatten().mean():.5f}")
                print(f" |- Curr: {current_discrete_text}")

            if config.wandb_logging:
                wandb.log({
                    "iteration": i,
                    "relaxed_loss": save_loss,
                    "avg_discrete_loss": avg_best_loss.item(),
                    "avg_best_loss": avg_best_loss.item(),
                    "current_entropy": current_entropy,
                    "learning_rate": scheduler.get_last_lr()[0],
                })
            del activations_discrete, discrete_loss # release memory

    return best_loss, best_discrete

In [7]:
def main():
    config = Config()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    if config.wandb_logging and not config.use_optuna:
        wandb.init(project=PROJECT, config=dataclasses.asdict(config))

    print("[+] Loading model and tokenizer...")
    # instanciate model
    t_model, t_toke = get_model_wrapper(config.model_id, 6)
    # load in desired loss function
    loss_e = return_maximise_logit_entropy_distribution(config.model_id)

    if config.use_optuna:
        print("[+] Using Optuna ...")
        study = optuna.create_study(
            study_name=config.optuna_study_name,
            storage=config.optuna_storage,
            direction="minimize",
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10),
        )
        study.optimize(
            lambda trial: attack(t_model, t_toke, adapt_for_optuna(config, trial))[0],
            n_trials=config.optuna_trials,
        )
        print("Best trial:")
        trial = study.best_trial
        print("  Value: ", trial.value)
        print("  Params: ")
        for key, value in trial.params.items():
            print("    {}: {}".format(key, value))
    else:
        print("[+] Start Attack ...")
        loss, best_prompt = attack(t_model, t_toke, config, loss_e, verbose=config.wandb_logging, discrete_loss_sample_rate=1)
        print()
        print("[+] Done. Final loss:", loss.mean())
        print("[*] Done. Example best prompt:", t_toke.decode(best_prompt[0]))  # Decode only the first batch item
        print()

    if config.wandb_logging:
        wandb.finish()

    return best_prompt

In [8]:
if __name__ == "__main__":
    torch.set_float32_matmul_precision("high")
    main()
# 3e49204e93ec17121e9c5a5d1021cc08b5c14d09

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: vedang. Use `wandb login --relogin` to force relogin


[+] Loading model and tokenizer...
[+] Start Attack ...


  0%|          | 0/500 [00:00<?, ?it/s]

[1] L-rel: 19.55417 / L-dis: 19.12102 / Best: 19.12102
 |- Curr: [' Casesrise seriousotechnologyAugust Fuckrd Interestingly redund darling downgrade Vista productive', 'ULToberifies comparisonTOlivion 163ques Valencia Book tastingtube plurality', ' horrifyingkillingscriptioneps Hardingwiceventsserviceimum Overs Fist angledATCH']


...

[57] L-rel: 18.85143 / L-dis: 19.06616 / Best: 18.79609
 |- Curr: ["olsrise restraints braces Vac Circuit Circuit cycle RISsc nipple probes':", ' Nations World Planetary StellarTO Kind�IONS repro Book may board status', ' Remastered Nationsocracyilogy AddictionIONSION Cart Serv drinking version VERS heck']


 12%|█▏        | 58/500 [00:26<03:16,  2.24it/s]

[58] L-rel: 18.83123 / L-dis: 19.07222 / Best: 18.79609
 |- Curr: ["olsrise restraints braces Vac Circuit Circuit cycle RISscelectric Contact':", ' Nations World Planetary StellarTO Kind�ers repro Book may boardimm', ' Remastered Nationsocracyocracy AddictionIONSION Cart Serv data version PART heck']


 12%|█▏        | 58/500 [00:26<03:23,  2.17it/s]


KeyboardInterrupt: 

In [ ]:
# import time

# time.sleep(64)

# from google.colab import runtime
# runtime.unassign()